In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

from pol_analysis.polData import PolData
from pol_analysis.runPhot import RunPhot

<frozen importlib._bootstrap>:241: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 96 from PyObject


In [ ]:
def run_all(obj, band, star_coords, stddev_0_vals=None, bhd_coords=(1021.5, 70.), show_seeing_plots=False, pdata_force_new=False, phot_force_new=False, special_ob_pairs=None, only_seeing_fit=False, centroid_box_size=5, pdata_filenames_for_masks=None):

    #Star coords for PSF measurements
    star_ex_ref, star_ey_ref = star_coords

    #BHD coords
    bhd_ex_ref, bhd_ey_ref = bhd_coords

    #Create the data object. This will run the mask creation and the cosmic ray rejection.  
    bhd = PolData(obj, band, filenames_for_mask_creation=pdata_filenames_for_masks, force_new=pdata_force_new)

    #Create the photometry object and run the common steps. 
    phot = RunPhot(bhd, force_new=phot_force_new)
    phot.subtract_background(box_size=(25,25))
    phot.get_source_positions(box_size=centroid_box_size)
    #Now, go through all the OB pairs.
    ob_combs = list()
    for ob_pair in bhd.ob_pairs:
        ob_combs.append([[ob_pair[0]],[ob_pair[1]]])
    if len(bhd.ob_pairs)>1:
        ob_combs.append([None, None])

    #Add special combinations if provided. 
    if special_ob_pairs is not None:
        ob_combs.extend(special_ob_pairs)

    for k, ob_comb in enumerate(ob_combs):
        ob_ids = ob_comb[0]
        mjds = ob_comb[1]
        #print(ob_ids, mjds)
        #continue

        #Get the seeing. 
        fnames = bhd.list_of_filenames(ob_ids=ob_ids, mjds=mjds)
        if stddev_0_vals is not None:
            stddev_0 = stddev_0_vals[k]
        else:
            stddev_0 = np.ones(len(fnames))*1.1
        phot.find_seeing(star_ex_ref, star_ey_ref, stddev_0=stddev_0, show_plots=show_seeing_plots, ob_ids=ob_ids, mjds=mjds)
        #print(phot.seeing)
        if only_seeing_fit:
            continue
